# Stock Market Reaction & Portfolio Management
**Author**: Moch Nabil Farras Dhiya

**E-mail**: nabilfarras923@gmail.com

**Institution**: Bandung Institute of Technology

**Student ID**: 10120034


---

**About**: This is a side-project created in order to analyze whether there is a relationship between COVID-19, Currency Exchange, and Stock Market, especially in Indonesia. The steps are of the followings:


1.   Extracting data from External Sources (Yahoo Finance) and Local MySQL DB (soon changed to COVID-19 API)
2.   Transform the data (making sure it is usable and consistent)
3.   Analyze the data trend and perform hypothesis testing to check the relationship between the data
4.   TSA forecasting to predict the stock price, using several models including, but not limited to ARIMA and Long Short-Term Memory
5.   Portfolio Optimization using Markowitz Efficient Frontier model 

These steps will be automatically implemented on daily basis, so we only need to monitor the dashboard.

# Import Modules

In [1]:
# Connect with local machine
import os

# Extracting file from URL (Online website)
import requests
from bs4 import BeautifulSoup

# Extract stock market data
import yfinance as yf

# Importing and transforming file
import pandas as pd

# Data manipulation
import numpy as np
import re # Cleaning texts
import time
import datetime as dt # Datetime manipulation
from scipy.stats import pearsonr # Statistics
from scipy.optimize import minimize # Minimze SR

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Time Series Analysis
import itertools
import pmdarima as pm # Automate ARIMA steps
from statsmodels.api import qqplot # QQPlot
from statsmodels.tsa.seasonal import seasonal_decompose # TSA Decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # Plot ACF & PACF Model
from statsmodels.tsa.stattools import adfuller # Stationary data
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch # Lagrange Multiplier test
from statsmodels.tsa.statespace.sarimax import SARIMAX # Plot SARIMAX Model
from statsmodels.tsa.api import Holt # Holt Model
from arch import arch_model # GARCH Model
from sklearn.metrics import mean_squared_error

# CNN Model
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D

# COVID19 Dataset
from covid19dh import covid19

# # Connect Colab with Local MySQL Database and delete existing table
# import sqlalchemy as sql
# from sqlalchemy.ext.declarative import declarative_base

# # Notify is there is an error to email
# import logging
# import logging.handlers

# Ignore warnings
import warnings

In [2]:
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Extract COVID-19 Daily Cases Data

As we already have a COVID-19 Daily Case data in our local database from previous projects, then we only need to fetch the data from our local database and make some modification.

In [3]:
covid_data, src = covid19('Indonesia', verbose = False)
covid_data

c:\Users\c640\AppData\Local\Programs\Python\Python310\lib\site-packages\covid19dh\main.py:107: UserWarning: error to fetch data
  warnings.warn("error to fetch data")


In [4]:
covid_data = covid_data.rename(columns = {'date': 'date_reported',
                                          'confirmed': 'new_cases'})
covid_data = covid_data[['date_reported', 'new_cases']].set_index('date_reported')
covid_data = (covid_data - covid_data.shift(1)).fillna(0)
covid_data = covid_data[: '2023-02-28'].reset_index()
covid_data

AttributeError: 'NoneType' object has no attribute 'rename'

## Connect to MySQL 'COVID-19' Local DB

In [ ]:
# print("================================")
# print("Connecting to Local DB .....")

# # Credentials to database connection
# hostname = "localhost"
# dbname = "covid_19"
# uname = os.getenv('DB_UNAME')
# pwd = os.getenv('DB_PASSWORD')

# # Create SQLAlchemy engine to connect to MySQL Database
# sqlEngine = sql.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host = hostname,
#                                                                                db = dbname,
#                                                                                user = uname,
#                                                                                pw = pwd))

# dbConnection = sqlEngine.connect()

# print("Successfully to Local DB.")
# print("================================")
# print()

## Fetch COVID-19 Daily Case Data

In [ ]:
# # Fetch only Indonesia data
# sql = "SELECT * FROM daily_case_death WHERE country = 'Indonesia'"
# covid_data = pd.read_sql(sql, 
#                          con = sqlEngine)
# covid_data

In [ ]:
covid_data.info()

As it comes from our local DB (loaded from previous projects which involves ETL process), we can be sure that this dataframe is already clean and ready to be analyzed.

# Extract Currency Exchange Data

We will extract the desired data from Yahoo Finance official website.

In [ ]:
# Start and End Date
start_date = '2017-01-01'
end_date = '2023-02-28'
# end_date = dt.datetime.now().strftime('%Y-%m-%d')

## IDR/USD

In [ ]:
# IDR/USD
idr_usd_ticker = yf.Ticker("IDR%3DX")
    
# get historical market data
init_idr_usd = idr_usd_ticker.history(period = "max")
init_idr_usd = init_idr_usd[start_date:]
init_idr_usd.info()

In [ ]:
init_idr_usd = init_idr_usd.iloc[:, :4]
init_idr_usd

## IDR/JPY

In [ ]:
# IDR/JPY
idf_jpy_ticker = yf.Ticker("JPYIDR=X")
    
# get historical market data
init_idr_jpy = idf_jpy_ticker.history(period = "max")
init_idr_jpy = init_idr_jpy[start_date:]
init_idr_jpy.info()

In [ ]:
init_idr_jpy = init_idr_jpy.iloc[:, :4]
init_idr_jpy

# Extract Stock Market Price Data

We will extract 2 stock market prices (BBCA.JK and ^JKSE) using yfinance module.

### BBCA.JK

In [ ]:
init_bbca_info = yf.download('BBCA.JK', 
                             start = start_date, 
                             end = end_date, 
                             progress = False)

init_bbca_info = init_bbca_info.reset_index()
init_bbca_info

In [ ]:
init_bbca_info

### ^JKSE

In [ ]:
init_jkse_info = yf.download('^JKSE', 
                             start = start_date, 
                             end = end_date, 
                             progress = False)

init_jkse_info = init_jkse_info.reset_index()
init_jkse_info.info()

In [ ]:
init_jkse_info

# Data Manipulation

We will do 3 things:
1. Expand and collapse the datasets so that it started from 2017-01-03
2. Tidy up the column name format, i.e. replacing space with _ and use lower alphabetical format
3. Change int datatype into float in stock price dataframe, and object into datetime in covid-19 dataframe
4. Make sure there is no dirty data, i.e. negative values
5. Make copy of the finance data, and merge it by interpolating. As we notice that there is always a 2 day gap every 5 day, possible happen because the data is not recorded in weekends.

## Expand and Collapse Datasets

In [ ]:
# Expand COVID dataset
covid_data_expanded = covid_data.append(pd.DataFrame({'date_reported': pd.date_range(start = '2017-01-03', 
                                                                                     end = '2020-01-21')}))
covid_data_expanded = covid_data_expanded.sort_values(by = 'date_reported').fillna(0)

# Collapse IDR/USD dataset
init_idr_usd = init_idr_usd.iloc[1:].reset_index()

# Collapse IDR/JPY dataset
init_idr_jpy = init_idr_jpy.iloc[1:].reset_index()

# Collapse BBCA.JK dataset
init_bbca_info = init_bbca_info.iloc[1:].reset_index()

## Column Formatting

In [ ]:
# Adjust the column names to sql readable format
def remove_space(text):
    text = text.strip()
    text = text.replace(' ', '_')
    return text

# Notice that all of the currency exchange and stock prices dataframe above have the same format
# so, we can tidy them all at once by looping
for df in [init_idr_usd, init_idr_jpy, init_bbca_info, init_jkse_info]:
    temp_target = []
    for col in df.columns:
        temp_target.append(remove_space(col.lower()))
        
    temp_dct = dict(zip(df.columns, temp_target))
    df = df.rename(columns = temp_dct, 
                   inplace = True)

## Casting

In [ ]:
# Stock market price dataframe
for df in [init_idr_usd, init_idr_jpy, init_bbca_info, init_jkse_info]:
    for col in df.columns:
        if df[col].dtypes == 'int64':
            df[col] = df[col].astype('float')
            

# COVID-19 dataframe
covid_data['date_reported'] = covid_data['date_reported'].apply(pd.to_datetime)

## Negative Value(s) Handling

In [ ]:
# Convert any negative values into positive, if there is any
for df in [init_idr_usd, init_idr_jpy, init_bbca_info, init_jkse_info]:
    for col in df.columns:
        if (df[col].dtypes == 'int64') or (df[col].dtypes == 'float64'):
            df[col] = df[col].apply(lambda x: x if x >= 0 else -x)

## Fix the Finance Datasets

In [ ]:
init_idr_usd

### IDR/USD

In [ ]:
# Create the dataframe
idr_usd = pd.DataFrame()

idr_usd['date'] = pd.date_range(start = '2017-01-03', 
                                end = end_date)

# Merge the data
idr_usd = pd.merge(idr_usd, init_idr_usd, how = 'left', on = 'date')
idr_usd

### IDR/JPY

In [ ]:
# Create the dataframe
idr_jpy = pd.DataFrame()

idr_jpy['date'] = pd.date_range(start = '2017-01-03',
                                end = end_date)

# Merge the data
idr_jpy = pd.merge(idr_jpy, init_idr_jpy, how = 'left', on = 'date')
idr_jpy

### BBCA.JK

In [ ]:
# Create the dataframe
bbca_info = pd.DataFrame()

bbca_info['date'] = pd.date_range(start = '2017-01-03', 
                                  end = end_date)

# Merge the data
bbca_info = pd.merge(bbca_info, init_bbca_info, how = 'left', on = 'date')
bbca_info

### ^JKSE

In [ ]:
# Create the dataframe
jkse_info = pd.DataFrame()

jkse_info['date'] = pd.date_range(start = '2017-01-03', 
                                  end = end_date)

# Merge the data
jkse_info = pd.merge(jkse_info, init_jkse_info, how = 'left', on = 'date')
jkse_info

# Missing Value(s) Handling

## COVID-19

In [ ]:
# Original COVID Dataset

missing_col = pd.DataFrame()
col_names = []
missing_value_percentage = []

for col in covid_data.columns:
    percentage = 100*len(covid_data.loc[(covid_data[col].isna()) 
                                        | (covid_data[col] == '-')])/len(covid_data)

    if percentage != 0:
        col_names.append(col)
        missing_value_percentage.append(percentage)
        
missing_col['col_names'] = col_names
missing_col['missing_value_percentage'] = missing_value_percentage

missing_col = missing_col.sort_values(by = 'missing_value_percentage').reset_index(drop = True)
missing_col

In [ ]:
# Expanded COVID Dataset

missing_col = pd.DataFrame()
col_names = []
missing_value_percentage = []

for col in covid_data.columns:
    percentage = 100 * len(covid_data_expanded.loc[(covid_data_expanded[col].isna()) 
                                                 | (covid_data_expanded[col] == '-')])/len(covid_data_expanded)

    if percentage != 0:
        col_names.append(col)
        missing_value_percentage.append(percentage)
        
missing_col['col_names'] = col_names
missing_col['missing_value_percentage'] = missing_value_percentage

missing_col = missing_col.sort_values(by = 'missing_value_percentage').reset_index(drop = True)
missing_col

## Currency Exchange

### IDR/USD

In [ ]:
missing_col = pd.DataFrame()
col_names = []
missing_value_percentage = []

for col in idr_usd.columns:
    percentage = 100*len(idr_usd.loc[(idr_usd[col].isna()) 
                                     | (idr_usd[col] == '-')])/len(idr_usd)

    if percentage != 0:
        col_names.append(col)
        missing_value_percentage.append(percentage)
        
missing_col['col_names'] = col_names
missing_col['missing_value_percentage'] = missing_value_percentage

missing_col = missing_col.sort_values(by = 'missing_value_percentage').reset_index(drop = True)
missing_col

### IDR/JPY

In [ ]:
missing_col = pd.DataFrame()
col_names = []
missing_value_percentage = []

for col in idr_jpy.columns:
    percentage = 100*len(idr_jpy.loc[(idr_jpy[col].isna()) 
                                     | (idr_jpy[col] == '-')])/len(idr_jpy)

    if percentage != 0:
        col_names.append(col)
        missing_value_percentage.append(percentage)
        
missing_col['col_names'] = col_names
missing_col['missing_value_percentage'] = missing_value_percentage

missing_col = missing_col.sort_values(by = 'missing_value_percentage').reset_index(drop = True)
missing_col

## Stock Market Price

### BBCA.JK

In [ ]:
missing_col = pd.DataFrame()
col_names = []
missing_value_percentage = []

for col in bbca_info.columns:
    percentage = 100*len(bbca_info.loc[(bbca_info[col].isna()) 
                                       | (bbca_info[col] == '-')])/len(bbca_info)

    if percentage != 0:
        col_names.append(col)
        missing_value_percentage.append(percentage)
        
missing_col['col_names'] = col_names
missing_col['missing_value_percentage'] = missing_value_percentage

missing_col = missing_col.sort_values(by = 'missing_value_percentage').reset_index(drop = True)
missing_col

### ^JKSE

In [ ]:
missing_col = pd.DataFrame()
col_names = []
missing_value_percentage = []

for col in jkse_info.columns:
    percentage = 100*len(jkse_info.loc[(jkse_info[col].isna()) 
                                       | (jkse_info[col] == '-')])/len(jkse_info)

    if percentage != 0:
        col_names.append(col)
        missing_value_percentage.append(percentage)
        
missing_col['col_names'] = col_names
missing_col['missing_value_percentage'] = missing_value_percentage

missing_col = missing_col.sort_values(by = 'missing_value_percentage').reset_index(drop = True)
missing_col

### Imputation

Notice that there are null values in these datasets and that the missing value percentage is quite high (>25%) due to the inactiveness in the weekends. Even so, we will try to do imputation on the characteristics of the data as we want to analyze it.

Since the data is a **TS numerical data**, then we will do **mean** based on the **nearest non-null value**.

In [ ]:
# IDR/USD dataset
a = idr_usd.rolling(3).mean()
b = idr_usd.iloc[::-1].rolling(3).mean()

c = a.fillna(b).fillna(idr_usd).interpolate(method = 'nearest').ffill().bfill()

idr_usd = idr_usd.fillna(c)

In [ ]:
# IDR/JPY dataset
a = idr_jpy.rolling(3).mean()
b = idr_jpy.iloc[::-1].rolling(3).mean()

c = a.fillna(b).fillna(idr_jpy).interpolate(method = 'nearest').ffill().bfill()

idr_jpy = idr_jpy.fillna(c)

In [ ]:
# BBCA.JK dataset
a = bbca_info.rolling(3).mean()
b = bbca_info.iloc[::-1].rolling(3).mean()

c = a.fillna(b).fillna(bbca_info).interpolate(method = 'nearest').ffill().bfill()

bbca_info = bbca_info.fillna(c)

In [ ]:
# ^JKSE dataset
a = jkse_info.rolling(3).mean()
b = jkse_info.iloc[::-1].rolling(3).mean()

c = a.fillna(b).fillna(jkse_info).interpolate(method = 'nearest').ffill().bfill()

jkse_info = jkse_info.fillna(c)

# Pattern Recognition

## COVID-19 Daily Cases

As we only interested in the daily cases of COVID-19, we can drop the rest of the columns.

In [ ]:
# COVID-19 daily cases plot
fig, ax = plt.subplots(constrained_layout = True)
covid_data_expanded.set_index('date_reported').plot(ax = ax, 
                                                    figsize = (18,12))

# First date COVID discovery in Indonesia
ax.axvline(x = "2020-03-02", 
           color = 'blue', 
           linestyle = '--')

# 2020 Eid Al-Fitr
ax.axvline(x = "2020-05-23", 
           color = 'purple', 
           linestyle = '--') 

# Indonesia's Independence Day
ax.axvline(x = "2020-08-17", 
           color = 'green', 
           linestyle = '--') 

# 2021 New Year
ax.axvline(x = "2021-01-01", 
           color = 'brown', 
           linestyle = '--')

# 2021 Eid Al-Fitr
ax.axvline(x = "2021-05-12", 
           color = 'purple', 
           linestyle = '--') 

# Indonesia's Independence Day
ax.axvline(x = "2021-08-17", 
           color = 'green', 
           linestyle = '--') 

# 2022 New Year
ax.axvline(x = "2022-01-01", 
           color = 'brown', 
           linestyle = '--') 

ax.set_ylabel("Daily cases")
ax.set_xlabel("Date")
ax.set_title('COVID-19 New Cases in Indonesia')

In [ ]:
# COVID-19 daily cases plot
fig, ax = plt.subplots(constrained_layout = True)
covid_data_expanded.set_index('date_reported')["2020-03-02":].plot(ax = ax, 
                                                                   figsize = (18,12))

# First date COVID discovery in Indonesia
ax.axvline(x = "2020-03-02", 
           color = 'blue', 
           linestyle = '--')

# 2020 Eid Al-Fitr
ax.axvline(x = "2020-05-23", 
           color = 'purple', 
           linestyle = '--') 

# Indonesia's Independence Day
ax.axvline(x = "2020-08-17", 
           color = 'green', 
           linestyle = '--') 

# 2021 New Year
ax.axvline(x = "2021-01-01", 
           color = 'brown', 
           linestyle = '--')

# 2021 Eid Al-Fitr
ax.axvline(x = "2021-05-12", 
           color = 'purple', 
           linestyle = '--') 

# Indonesia's Independence Day
ax.axvline(x = "2021-08-17", 
           color = 'green', 
           linestyle = '--') 

# 2022 New Year
ax.axvline(x = "2022-01-01", 
           color = 'brown', 
           linestyle = '--') 

ax.set_ylabel("Daily cases")
ax.set_xlabel("Date")
ax.set_title('COVID-19 New Cases in Indonesia')

From the graphs above, we can see that:
1. There is a significant **increment** on daily cases, especially **2-4 weeks after New Year's Holiday**, because people's mobility during holiday is very high, and will most likely spread the virus.
2. Significant **increment** trend is also found not long **after Eid Fitri Holiday**, since the majority of Indonesian are Muslim, and usually people go back to their hometown on such occassion, making certain locations are crowded, and the virus will most likely to spread.

## Currency Exchange

In this analysis, we will use the 'Close' feature, as

1. 'Close' price reflects all of the information available to all market participants (especially institutional market participants who have more accurate information) at the end of trading the stock.
2. (especially for hedge funds or mutual fund managers) the close price is a determinant of the performance and wealth of investors for the day.
3. 'Close' price reflects the price position at which the investor dares to hold a position, in the face of all information that may occur at night, when there is no trade.
4. More than 90% of technical indicators used by technical analysts use close prices as their main input. This causes the position of the price to close, can trigger a buy signal or a sell signal.

Source : https://sahamology.id/arti-harga-open-harga-high-harga-low-dan-harga-close-dalam-analisis-teknikal/

In [ ]:
# IDR/USD
idr_usd = idr_usd[['date', 'close']]

# IDR/JPY
idr_jpy = idr_jpy[['date', 'close']]

In [ ]:
# Indonesia's Currency Exchange plot
fig, (ax1, ax2) = plt.subplots(2,1, constrained_layout = True)
idr_usd.set_index('date').plot(ax = ax1, 
                               figsize = (18,12))

idr_jpy.set_index('date').plot(ax = ax2, 
                               figsize = (18,12))

# First date COVID discovery
ax1.axvline(x = "2020-03-02", 
            color = 'blue', 
            linestyle = '--') # in Indonesia

ax2.axvline(x = "2020-03-02", 
            color = 'blue', 
            linestyle = '--') # in Indonesia

ax1.axvline(x = "2020-01-20", 
            color = 'black', 
            linestyle = '--') # in USA

ax2.axvline(x = "2020-01-16", 
            color = 'purple', 
            linestyle = '--') # in Japan

# Indonesia's Independence Day
ax1.axvline(x = "2020-08-17", 
            color = 'green', 
            linestyle = '--') 

ax2.axvline(x = "2020-08-17", 
            color = 'green', 
            linestyle = '--')

# 2021 New Year
ax1.axvline(x = "2021-01-01", 
            color = 'brown', 
            linestyle = '--') 

ax2.axvline(x = "2021-01-01", 
            color = 'brown', 
            linestyle = '--')

# Indonesia's Independence Day
ax1.axvline(x = "2021-08-17", 
            color = 'green', 
            linestyle = '--') 

ax2.axvline(x = "2021-08-17", 
            color = 'green', 
            linestyle = '--')

# 2022 New Year
ax1.axvline(x = "2022-01-01", 
            color = 'brown', 
            linestyle = '--') 

ax2.axvline(x = "2022-01-01", 
            color = 'brown', 
            linestyle = '--')

ax1.set_ylabel("Close")
ax2.set_ylabel("Close")

ax1.set_xlabel("Date")
ax2.set_xlabel("Date")

ax1.set_title('IDR/USD Curency Exchange 2017 - 2022')
ax2.set_title('IDR/JPY Currency Exchange 2017 - 2022')

It seems that there are outliers in the data, but we will **let them** as it is in this analysis. Since it is probably happen because of an **error in the system**.

### Post COVID-19 Discovery

In [ ]:
# Indonesia's Currency Exchange plot
fig, (ax1, ax2) = plt.subplots(2,1, constrained_layout = True)
idr_usd.set_index('date')["2020-01-01":].plot(ax = ax1, 
                                              figsize = (18,12))

idr_jpy.set_index('date')["2020-01-01":].plot(ax = ax2, 
                                              figsize = (18,12))

# First date COVID discovery
ax1.axvline(x = "2020-03-02", 
            color = 'blue', 
            linestyle = '--') # in Indonesia

ax2.axvline(x = "2020-03-02", 
            color = 'blue', 
            linestyle = '--') # in Indonesia

ax1.axvline(x = "2020-01-20", 
            color = 'black', 
            linestyle = '--') # in USA

ax2.axvline(x = "2020-01-16", 
            color = 'purple', 
            linestyle = '--') # in Japan

# Indonesia's Independence Day
ax1.axvline(x = "2020-08-17", 
            color = 'green', 
            linestyle = '--') 

ax2.axvline(x = "2020-08-17", 
            color = 'green', 
            linestyle = '--')

# 2021 New Year
ax1.axvline(x = "2021-01-01", 
            color = 'brown', 
            linestyle = '--') 

ax2.axvline(x = "2021-01-01", 
            color = 'brown', 
            linestyle = '--')

# Indonesia's Independence Day
ax1.axvline(x = "2021-08-17", 
            color = 'green', 
            linestyle = '--') 

ax2.axvline(x = "2021-08-17", 
            color = 'green', 
            linestyle = '--')

# 2022 New Year
ax1.axvline(x = "2022-01-01", 
            color = 'brown', 
            linestyle = '--') 

ax2.axvline(x = "2022-01-01", 
            color = 'brown', 
            linestyle = '--')

ax1.set_ylabel("Close")
ax2.set_ylabel("Close")

ax1.set_xlabel("Date")
ax2.set_xlabel("Date")

ax1.set_title('IDR/USD Curency Exchange 2017 - 2022')
ax2.set_title('IDR/JPY Currency Exchange 2017 - 2022')

# Stock Market Price

By the same argument as the 'Currency Exchange' section, we will also use the 'Close' feature in this analysis.

In [ ]:
# BBCA.JK
bbca_info = bbca_info[['date', 'close']]

# CMPP.JK
jkse_info = jkse_info[['date', 'close']]

In [ ]:
# Stock market price plot
fig, (ax1, ax2) = plt.subplots(2,1, constrained_layout = True)
bbca_info.set_index('date').plot(ax = ax1, 
                                 figsize = (18,12))

jkse_info.set_index('date').plot(ax = ax2, 
                                 figsize = (18,12))

# First date COVID discovery in Indonesia
ax1.axvline(x = "2020-03-02", 
            color = 'blue', 
            linestyle = '--') 

ax2.axvline(x = "2020-03-02", 
            color = 'blue', 
            linestyle = '--')

# Indonesia's Independence Day
ax1.axvline(x = "2020-08-17", 
            color = 'green',
            linestyle = '--') 

ax2.axvline(x = "2020-08-17",
            color = 'green', 
            linestyle = '--')

# 2021 New Year
ax1.axvline(x = "2021-01-01", 
            color = 'brown', 
            linestyle = '--') 

ax2.axvline(x = "2021-01-01", 
            color = 'brown', 
            linestyle = '--')

# Indonesia's Independence Day
ax1.axvline(x = "2021-08-17",
            color = 'green', 
            linestyle = '--') 

ax2.axvline(x = "2021-08-17", 
            color = 'green',
            linestyle = '--')

# 2022 New Year
ax1.axvline(x = "2022-01-01", 
            color = 'brown', 
            linestyle = '--') 

ax2.axvline(x = "2022-01-01", 
            color = 'brown', 
            linestyle = '--')

ax1.set_ylabel("Close")
ax2.set_ylabel("Close")

ax1.set_xlabel("Date")
ax2.set_xlabel("Date")

ax1.set_title('BBCA.JK Stock Price 2017 - 2022')
ax2.set_title('^JKSE Stock Price 2017 - 2022')

### Post COVID-19 Discovery

In [ ]:
# Stock market price post COVID plot
fig, (ax1, ax2) = plt.subplots(2,1, constrained_layout = True)
bbca_info.set_index('date')["2020-03-02":].plot(ax = ax1, 
                                                figsize = (18,12))

jkse_info.set_index('date')["2020-03-02":].plot(ax = ax2, 
                                                figsize = (18,12))

# First date COVID discovery in Indonesia
ax1.axvline(x = "2020-03-02", 
            color = 'blue', 
            linestyle = '--') 

ax2.axvline(x = "2020-03-02", 
            color = 'blue', 
            linestyle = '--')

# Indonesia's Independence Day
ax1.axvline(x = "2020-08-17", 
            color = 'green', 
            linestyle = '--') 

ax2.axvline(x = "2020-08-17", 
            color = 'green', 
            linestyle = '--')

# 2021 New Year
ax1.axvline(x = "2021-01-01", 
            color = 'brown', 
            linestyle = '--') 

ax2.axvline(x = "2021-01-01", 
            color = 'brown', 
            linestyle = '--')

# Indonesia's Independence Day
ax1.axvline(x = "2021-08-17",
            color = 'green', 
            linestyle = '--') 

ax2.axvline(x = "2021-08-17", 
            color = 'green',
            linestyle = '--')

# 2022 New Year
ax1.axvline(x = "2022-01-01",
            color = 'brown', 
            linestyle = '--') 

ax2.axvline(x = "2022-01-01",
            color = 'brown', 
            linestyle = '--')

ax1.set_ylabel("Close")
ax2.set_ylabel("Close")

ax1.set_xlabel("Date")
ax2.set_xlabel("Date")

ax1.set_title('BBCA.JK Stock Price post COVID-19 Discovery in Indonesia')
ax2.set_title('^JKSE Stock Price post COVID-19 Discovery in Indonesia')

From the graphs above, we can see that:
1. On the **first 3-4 weeks of COVID-19 discovery in Indonesia**, the price of BBCA.JK and ^JKSE stock market went **down** for **31.21%** and **30.31%**, respecively, and managed to recover since the first two weeks of April 2020.
2. **2-3 weeks after Indonesia's Independence day in 2020**, there was also a **decremental** trend for both stock market price, and it went down for **8.65%** and **7.62%** , respectively. But these stock prices recovered since October 2020.
3. Not long after **2021' New Year's Eve**, there is also a **decremental** trend for both stock market price, and it went down for **15.37%** and **3.42%**, respectively, and did not show any sign of recovery until October 2021.
4. Surprisingly, both stock market prices give an **incremental** trend starting from **October in 2020 and 2021**.

# Hypothesis Testing

We will see if any pair of these datasets correlate with each other by using hypothesis testing with 0.05 significance level.

## Economic Datasets

Notice that there are datasets which start from 2017-01-02, and there is also datasets which start from 2017-01-03. For consistency, we will drop the data on 2017-01-02 and start the index from 2017-01-03.

In [ ]:
idr_usd = idr_usd.set_index('date')['2017-01-03':]

idr_jpy = idr_jpy.set_index('date')['2017-01-03':]

bbca_info = bbca_info.set_index('date')['2017-01-03':]

jkse_info = jkse_info.set_index('date')['2017-01-03':]

In [ ]:
df = pd.DataFrame()
df.index = idr_usd.index
df['idr_usd'] = idr_usd.values
df['idr_jpy'] = idr_jpy.values
df['bbca_info'] = bbca_info.values
df['jkse_info'] = jkse_info.values

significance_level = 0.05
print('Significance level = 0.05 \n')

temp = pd.DataFrame()
features = []
correlations = []
p_values = []
conclusions = []
for col in df.columns:
    for sub_col in df.columns:
        if col != sub_col:
            r, p = pearsonr(df.dropna()[col], df.dropna()[sub_col])
            
            features.append(f'{col} vs. {sub_col}')
            correlations.append(r)
            p_values.append(p)
            
            if p < significance_level:
                conclusions.append('Reject H0')
                
            else:
                conclusions.append('Do not reject H0')

        else:
            pass
        
temp['features'] = features
temp['correlations'] = correlations
temp['p_values'] = p_values
temp['conclusions'] = conclusions

temp

Based on the hypothesis testing result, we can conclude that **IDR/USD** **moderately correlates** with both BBCA dataset, with pearson coefficient of 0.624731. In addition, BBCA dataset also **moderately correlates** with both ^JKSE dataset, with pearson coefficient of 0.612116, while the other pair datasets do not correlate with each other, using significance level of 0.05.

**Note**: IDR/USD and ^JKSE datasets do correlate with each other as its p-value is significant, but put in mind that its pearson coefficient is **close to 0**. And such, we will **treat** it as they **do not correlate** with each other.

## vs. COVID Data

In [ ]:
idr_usd

In [ ]:
covid_data_expanded = covid_data_expanded.set_index('date_reported')

In [ ]:
idr_usd_covid = idr_usd['2020-01-03':covid_data_expanded.index[-1].strftime('%Y-%m-%d')]
idr_usd_covid = idr_usd_covid.reset_index()

idr_jpy_covid = idr_jpy['2020-01-03':covid_data_expanded.index[-1].strftime('%Y-%m-%d')]
idr_jpy_covid = idr_jpy_covid.reset_index()

bbca_info_covid = bbca_info['2020-01-03':covid_data_expanded.index[-1].strftime('%Y-%m-%d')]
bbca_info_covid = bbca_info_covid.reset_index()

jkse_info_covid = jkse_info['2020-01-03':covid_data_expanded.index[-1].strftime('%Y-%m-%d')]
jkse_info_covid = jkse_info_covid.reset_index()

In [ ]:
idr_usd_covid

In [ ]:
df = pd.DataFrame()
df.index = covid_data_expanded['2020-01-03':].index
df['covid'] = covid_data_expanded['2020-01-03':]['new_cases'].values
df['idr_usd'] = idr_usd_covid['close'].values
df['idr_jpy'] = idr_jpy_covid['close'].values
df['bbca_info'] = bbca_info_covid['close'].values
df['jkse_info'] = jkse_info_covid['close'].values

significance_level = 0.05
print('Significance level = 0.05 \n')

temp = pd.DataFrame()
features = []
correlations = []
p_values = []
conclusions = []
for col in df.columns:
    for sub_col in df.columns:
        if col != sub_col:
            r, p = pearsonr(df.dropna()[col], df.dropna()[sub_col])
            
            features.append(f'{col} vs. {sub_col}')
            correlations.append(r)
            p_values.append(p)
            
            if p < significance_level:
                conclusions.append('Reject H0')
                
            else:
                conclusions.append('Do not reject H0')

        else:
            pass
        
temp['features'] = features
temp['correlations'] = correlations
temp['p_values'] = p_values
temp['conclusions'] = conclusions

temp

In [ ]:
df

# Model Identification

In this project, for simplicity purposes, we will only focus on performing TSA on the BBCA data. If you want to perform TSA on another data, then just repeat the process and apply it to the data you want to perform TSA.

In particular, these dataset is chosen since it is a financial data, so it has high volatility, and one of our aim here is to compare base model with ARCH/GARCH model. In addition, from the previous section, we know that BBCA dataset moderately/strongly correlate with other datasets, and we will use ARIMAX model with the other dataset as the exogenous arguments to predict the BBCA dataset.

## Check Stationary

To perform TSA, the data itself **has to be a stationary data** as its statistical properties do not change over time. The assumption in time series data modeling is that each observation is independent of each other.

In [ ]:
bbca_info = bbca_info['2020-04-01':]

In [ ]:
plt.figure(figsize = (18,12))

additive_result = seasonal_decompose(bbca_info.values, model = 'additive', period = 1)
additive_result.plot()
plt.show()

In [ ]:
plt.figure(figsize = (18,12))

multiplicative_result = seasonal_decompose(bbca_info.values, model = 'multiplicative', period = 1)
multiplicative_result.plot()
plt.show()

In particular, we had observe from the the graph above that our dataset has **increment trend** (additive, in particular, as the multiplicative residual are high. But we will see this in further section). Thus, it **should be not stationer**. We will check this claim by **ADF-Test using significance level 95%** ($\alpha$ = 0.05) and from its **ACF/PACF Plot**.

In [ ]:
# Test
test = adfuller(bbca_info['close'])

# Statistic result
print('Augmented-Dickey Fuller Result :', test[0])

# p-value
print('p-value :', test[1])

In [ ]:
# Create figure
fig, (ax1, ax2) = plt.subplots(2, 1, 
                               figsize = (18,12))
 
# ACF Plot from stock market price data
plot_acf(bbca_info, 
         lags = 15, 
         zero = False,
         ax = ax1)

# PACF Plot from stock market price data
plot_pacf(bbca_info, 
          lags = 15, 
          zero = False, 
          ax = ax2)

plt.show()

Based on the hypothesis testing result above, we can conclude that the BBCA dataset is **not stationary**, as its **p-value** is **higher** than the $\alpha$. And then, also notice that it has **no significant** change in the **ACF** plot. So, we will perform **differentiation** on this dataset.

## Convert into Stationer Data

If the data is not stationary, it is necessary to transform the data. One of the transformation methods is to use the **differentiation method**.

In [ ]:
# Differentiation
bbca_info_diff = bbca_info.diff().dropna()

# Differentiation result plotting
bbca_info_diff.plot()
plt.show()

In [ ]:
# Adfuller Test
test_1 = adfuller(bbca_info_diff['close'])

# Statistic result
print('Augmented-Dickey Fuller Result :', test_1[0])

# p-value
print('p-value :', test_1[1])

Since the p-value is already lower than the $\alpha$, then it means that the BBCA dataset is now stationary.

# Model Estimation

To decide the order of p and q in the ARIMA model, i.e. estimating the most suitable model for the data, there are 2 steps that we can do:

1. **Manual**: Analyze the ACF and PACF plot and look at which lag the cut off happen to determine the p and q then look for the least AIC and BIC model.

2. **Auto**: Use automated iterative steps to look for the model with the least AIC for the given range of p and q using pmdarima package.

For simplicity, here we will only use the **Auto method** and will only look for the least **AIC** model. Here, AIC criteria is chosen because in financial datasets, the volatility is high, and considering the **unpredicted external factors** (e.g. COVID-19), there will **not** be a **fixed/final model** in such cases.

## Automation Method (Auto-ARIMA)

### Best Model Estimation

In [ ]:
# Get best ARIMA model with lowest AIC
results_auto_arima_bbca = pm.auto_arima(bbca_info, 
                                        start_p = 0,
                                        start_q = 0,
                                        max_p = 10,
                                        max_q = 10,
                                        d = None,
                                        trace = True,
                                        error_action = 'ignore')

results_auto_arima_bbca.summary()

Based on the summary table above, we can see that the **ARIMA(3,1,1)** and **ARIMA(4,1,0)** have the **lowest AICs**. Thus, we will try to analyze the performance of that model by using **diagnostic test**. If it is not suited to be a predictor model, we will pick another model (iteratively).

### ARIMA(3,1,1)

In [ ]:
model_1a_bbca = SARIMAX(bbca_info, 
                        order = (3,1,1))

results_1a_bbca = model_1a_bbca.fit()

results_1a_bbca.summary()

**Conclusion**

----

Notice that even without doing diagnostic test (i.e. Ljung Box Test), we can already see that the **parameters** of this model are **not statistically significant**. Thus, we will not use this model for our forecasting task.

### ARIMA(4,1,0)

In [ ]:
model_1b_bbca = SARIMAX(bbca_info, 
                        order = (4,1,0))

results_1b_bbca = model_1b_bbca.fit()

results_1b_bbca.summary()

In [ ]:
arima_resid = results_1b_bbca.resid
white_noise_arima = acorr_ljungbox(arima_resid ** 2, lags = [10], return_df = True)
white_noise_arima

**Diagnostic Test & Conclusion**

**Ljung-Box**

----

$H_0$ = Error term is white noise

$H_1$ = Error term is not white noise

Based on the table above, get that the p-value for Ljung-Box statistical test is 0.01. Since p-value = 0.01 < 0.05 = $\alpha$, we **reject our null hypothesis**. Thus, we conclude that our **residuals** are **not independently distributed** using significance level of 0.05. 

**Conclusion**

----

Based on the hypothesis testing and the model summary above, we can conclude that the residual from our ARIMA(4,1,0) is **not white noise**, based on significance level of 0.05. Thus, we **will** conduct GARCH to fit our model.

### Final Model

In [ ]:
arima_model = SARIMAX(bbca_info, 
                      order = (4,1,0)).fit()

arima_resid = arima_model.resid

## ARIMAX Model

Next, since we know that BBCA datasets moderately/strongly correlate with other datasets, we will try to use the other as the exogenous arguments to predict our BBCA dataset. In this case, we will use **only IDR/USD** dataset as our **exogenous argument**. JKSE dataset is not included since both are stock market dataset, and it is not a wise choice to use dataset with the same 'type' as the exogenous argument. In the other hand, COVID dataset is not used since it has different starting point, and if we want to use it as our exogenous argument, we have to cut our BBCA dataset, which will cause a huge loss information, not considering that COVID is only temporary.

In [ ]:
# Fetch other datasets and 'merge' it into BBCA dataset
exog_idr_usd = pd.DataFrame()
exog_idr_usd.index = bbca_info.index
exog_idr_usd['close'] = idr_usd['2020-04-01':]['close']

exog_idr_usd

### Best Model Estimation

In [ ]:
# Get best ARIMAX model with lowest AIC
results_auto_arimax_bbca = pm.auto_arima(bbca_info, 
                                         start_p = 0,
                                         start_q = 0,
                                         max_p = 10,
                                         max_q = 10,
                                         d = None,
                                         exog = exog_idr_usd.values,
                                         trace = True,
                                         error_action = 'ignore')

results_auto_arimax_bbca.summary()

Based on the summary table above, we can see that as the previous ones, the **ARIMAX(3,1,1,'IDR/USD')** and **ARIMAX(4,1,0,'IDR/USD')** have the **lowest AICs**. Thus, we will try to analyze the performance of that model by using **diagnostic test**. If it is not suited to be a predictor model, we will pick another model (iteratively).

### ARIMAX(3,1,1,'IDR/USD')

In [ ]:
# Model ARIMAX(2,1,2,'IDR/USD')
model_2a_bbca = SARIMAX(bbca_info, 
                        order = (3,1,1),
                        exog = exog_idr_usd.values)

results_2a_bbca = model_2a_bbca.fit()

results_2a_bbca.summary()

**Conclusion**

----

Notice that even without doing diagnostic test (i.e. Ljung-Box Test), we can already see that the **parameters** of this model are **not statistically significant**. Thus, we will not use this model for our forecasting task.

### ARIMAX(4,1,0,'IDR/USD')

In [ ]:
# Model ARIMAX(4,1,0,'IDR/USD')
model_2b_bbca = SARIMAX(bbca_info, 
                       order = (4,1,0),
                       exog = exog_idr_usd.values)

results_2b_bbca = model_2b_bbca.fit()

results_2b_bbca.summary()

In [ ]:
arimax_resid = results_2b_bbca.resid
white_noise_arimax = acorr_ljungbox(arimax_resid ** 2, lags = [10], return_df = True)
white_noise_arimax

**Diagnostic Test & Conclusion**

**Ljung-Box**

----

$H_0$ = Error term is white noise

$H_1$ = Error term is not white noise

Based on the table above, get that the p-value for Ljung-Box statistical test is 0.01. Since p-value $\approx$ 0.01 < 0.05 = $\alpha$, we **reject our null hypothesis**. Thus, we conclude that our **residuals** are **not independently distributed** using significance level of 0.05. 

**Conclusion**

----

Based on the hypothesis testing and the model summary above, we can conclude that the residual from our ARIMAX(4,1,0,'IDR/USD') is **not white noise**, based on significance level of 0.05. Thus, we **will** conduct GARCH to fit our model.

### Final Model

In [ ]:
arimax_model = SARIMAX(bbca_info, 
                       order = (4,1,0),
                       exog = exog_idr_usd.values).fit()

arimax_resid = arimax_model.resid

## SARIMA Model

From the analysis in the previous section, we know that **BBCA** stock price dataset have an **addition trend** and a **large entries/records**. As such, we will use **SARIMA** model to **train** our data instead of Holt or SES model. For simplicity, we will use a grid search to iteratively explore different combinations of parameters.

### Best Model Estimation

In [ ]:
def sarima_grid_search(data, seasonal_period):
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], seasonal_period) for x in list(itertools.product(p, d, q))]
    
    mini = 1e6
    
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = SARIMAX(data,
                              order = param,
                              seasonal_order = param_seasonal,
                              enforce_stationarity = False,
                              enforce_invertibility = False)

                results = mod.fit()
                
                if results.aic < mini:
                    mini = results.aic
                    param_mini = param
                    param_seasonal_mini = param_seasonal

            except:
                continue
                
    print(f'The set of parameters with the minimum AIC is: SARIMA{param_mini}x{param_seasonal_mini} - AIC:{mini}')

Since we are observing by month, then pick **m = 12**.

In [ ]:
sarima_grid_search(bbca_info, 12)

Based on the summary table above, we can see that the **SARIMA(1,1,1)x(0,1,1,12)** has the **lowest AIC**. Thus, we will try to analyze the performance of that model by using **diagnostic test**. If it is not suited to be a predictor model, we will pick another model (iteratively).

In [ ]:
model_3_bbca = SARIMAX(bbca_info, 
                       order = (1,1,1),
                       seasonal_order = (0,1,1,12),
                       enforce_stationarity = False,
                       enforce_invertibility = False)

results_3_bbca = model_3_bbca.fit()

results_3_bbca.summary()

In [ ]:
sarima_resid = results_3_bbca.resid
white_noise_sarima = acorr_ljungbox(sarima_resid ** 2, lags = [10], return_df = True)
white_noise_sarima

**Diagnostic Test & Conclusion**

**Ljung-Box**

----

$H_0$ = Error term is white noise

$H_1$ = Error term is not white noise

Based on the table above, get that the p-value for Ljung-Box statistical test is ~0.05. Since p-value $\approx$ 0.05 = $\alpha$, we **fail to reject our null hypothesis**. Thus, we conclude that our **residuals** are **independently distributed** using significance level of 0.05. 

**Conclusion**

----

Based on the hypothesis testing and the model summary above, we can conclude that the residual from our SARIMA(1,1,1)x(0,1,1,12) is **white noise**, based on significance level of 0.05.

### Final Model

In [ ]:
# Model SARIMA(1,1,1)x(0,1,1,12)
sarima_model = SARIMAX(bbca_info, 
                       order = (1,1,1),
                       seasonal_order = (0,1,1,12),
                       enforce_stationarity = False,
                       enforce_invertibility = False).fit()

sarima_resid = sarima_model.resid

## GARCH Model

Since we are dealing with a **financial data** which has **high volatility**, then it is a good idea to analyze and predict the future values using **GARCH model**. Here, GARCH is chosen instead of ARCH because in general, it has much **less parameters** and **performs better** than the ARCH model. The generalized autoregressive conditional heteroskedasticity (GARCH) model has only **three parameters** that allow for an **infinite number of squared roots** to influence the conditional variance.

In addition to that, it was seen that our **previous models** have a sign of having a **GARCH effect**. Thus, we will test the performance of our previous models combined by the GARCH effect for both models (ARIMA and ARIMAX).

### ARIMA(4,1,0)

Even though we have noticed that ARIMA's residuals is not random from Ljung-Box test, we **can not conclude yet** that there exists **heteroskedasticity** effect. Thus, we will perform LM test to check it.

$H_0$： ARCH Effect not exists

$H_1$：ARCH Effect exists

In [ ]:
LM_pvalue = het_arch(arima_resid, ddof = 4)[1]
print(f'LM-test-Pvalue: {LM_pvalue}')

Since we got that p-value $\approx$ 0, then we **reject our null hypothesis**, and conclude that there exists a GARCH effect.

Now, we will **look** for the **p, q parameters** for the model **iteratively**, by observing the significancy of parameters of each possible values of p, q.

In [ ]:
mdl_garch = arch_model(arima_resid, vol = 'GARCH', p = 1, q = 1)
arima_garch = mdl_garch.fit()
arima_garch.summary()

We notice that the only model with **significant parameters** is **GARCH(1,1)**. Thus, we will combine it with our base ARIMA model.

**Residual Diagnosis**

In [ ]:
garch_std_resid = pd.Series(arima_garch.resid / arima_garch.conditional_volatility)
fig = plt.figure(figsize = (15, 8))

# Residual
garch_std_resid.plot(ax = fig.add_subplot(3,1,1), title = 'GARCH Standardized-Residual', legend = False)

# ACF/PACF
plot_acf(garch_std_resid, zero = False, lags = 40, ax = fig.add_subplot(3,2,3))
plot_pacf(garch_std_resid, zero = False, lags = 40, ax = fig.add_subplot(3,2,4))

# QQ-Plot & Norm-Dist
qqplot(garch_std_resid, line = 's', ax = fig.add_subplot(3,2,5)) 
plt.title("QQ Plot")
fig.add_subplot(3,2,6).hist(garch_std_resid, bins = 40)
plt.title("Histogram")

plt.tight_layout()
plt.show()

In [ ]:
white_noise_garch = acorr_ljungbox(garch_std_resid, lags = [10], return_df=True)
white_noise_garch

By the residual diagnostic tests above, we can conclude that the residual follows the normal distribution and is a white noise. Therefore, no explanatory variable can be taken.

### ARIMAX(4,1,0,'IDR/USD')

Even though we have noticed that ARIMA's residuals is not random from Ljung-Box test, we **can not conclude yet** that there exists **heteroskedasticity** effect. Thus, we will perform LM test to check it.

$H_0$： ARCH Effect not exists

$H_1$：ARCH Effect exists

In [ ]:
LM_pvalue = het_arch(arimax_resid, ddof = 4)[1]
print(f'LM-test-Pvalue: {LM_pvalue}')

Since we got that p-value $\approx$ 0.03 < 0.05 = $\alpha$, then we **reject our null hypothesis**, and conclude that there exists a GARCH effect.

Now, we will **look** for the **p, q parameters** for the model **iteratively**, by observing the significancy of parameters of each possible values of p, q.

In [ ]:
mdl_garchx = arch_model(arimax_resid, vol = 'GARCH', p = 1, q = 1)
arimax_garch = mdl_garchx.fit()
arimax_garch.summary()

We notice that the only model with **significant parameters** is **GARCH(1,1)**. Thus, we will combine it with our base ARIMAX model.

**Residual Diagnosis**

In [ ]:
garch_std_resid = pd.Series(arimax_garch.resid / arimax_garch.conditional_volatility)
fig = plt.figure(figsize = (15, 8))

# Residual
garch_std_resid.plot(ax = fig.add_subplot(3,1,1), title = 'GARCH Standardized-Residual', legend = False)

# ACF/PACF
plot_acf(garch_std_resid, zero = False, lags = 40, ax = fig.add_subplot(3,2,3))
plot_pacf(garch_std_resid, zero = False, lags = 40, ax = fig.add_subplot(3,2,4))

# QQ-Plot & Norm-Dist
qqplot(garch_std_resid, line = 's', ax = fig.add_subplot(3,2,5)) 
plt.title("QQ Plot")
fig.add_subplot(3,2,6).hist(garch_std_resid, bins = 40)
plt.title("Histogram")

plt.tight_layout()
plt.show()

In [ ]:
white_noise_garch = acorr_ljungbox(garch_std_resid, lags = [10], return_df=True)
white_noise_garch

By the residual diagnostic tests above, we can conclude that the residual follows the normal distribution and is a white noise. Therefore, no explanatory variable can be taken.

## CNN & LSTM

### 3-D Format Conversion

We need to convert the data into 3-D format so that it can be processed using CNN method.

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
    
        # find the end of this pattern
        end_ix = i + n_steps

        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break

        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    
    return np.array(X), np.array(y)

In [ ]:
n_steps = 7

X_bbca, y_bbca = split_sequence(bbca_info.values, n_steps)

In [ ]:
# transform input from [samples, features] to [samples, timesteps, features]
X_bbca = X_bbca.reshape((X_bbca.shape[0], X_bbca.shape[1], 1))
print("BBCA shape : ", X_bbca.shape)

### Split Dataset

In [ ]:
# split a univariate dataset into train/test sets
def split_dataset(data):
    # split into two dataset
    train, test = data[:int(0.75 * len(data))], data[int(0.75 * len(data)):]
    
    return train, test

In [ ]:
X_train_bbca, X_test_bbca = split_dataset(X_bbca)
y_train_bbca, y_test_bbca = split_dataset(y_bbca)

In [ ]:
print("BBCA Dataset")
print("X_train shape : ", X_train_bbca.shape)
print("X_test shape : ", X_test_bbca.shape)
print("y_train shape : ", y_train_bbca.shape)
print("y_test shape : ", y_test_bbca.shape)

### Change to Supervised

We will convert the TS data into Supervised data.

In [ ]:
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out = 7):
    # flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out

        # ensure we have enough data for this instance
        if out_end < len(data):
            x_input = data[in_start:in_end, 0]
            x_input = x_input.reshape((len(x_input), 1))
            X.append(x_input)
            y.append(data[in_end:out_end, 0])

        # move along one time step
        in_start += 1
    
    return np.array(X), np.array(y)

### Model

In [ ]:
# Train the model
def build_model(dl_model, train, n_input):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    
    # define parameters
    verbose, epochs, batch_size = 1, 100, 4
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

    # define model
    if dl_model == 'CNN':
        model = Sequential()
        model.add(Conv1D(filters = 16, 
                         kernel_size = 3, 
                         activation = 'relu',
                         input_shape = (n_timesteps, n_features)))
        model.add(MaxPooling1D(pool_size = 2))
        model.add(Flatten())
        model.add(Dense(10, 
                        activation = 'relu'))
        model.add(Dense(n_outputs))
        model.compile(loss = 'mse', 
                      optimizer = 'adam')

        # fit network
        model.fit(train_x, train_y, epochs = epochs, batch_size = batch_size, verbose = verbose)
        
        return model
        
    elif dl_model == 'LSTM':
        model = Sequential()
        model.add(LSTM(200, 
                       activation = 'relu', 
                       input_shape = (n_timesteps, n_features)))
        model.add(Dense(100, 
                        activation = 'relu'))
        model.add(Dense(n_outputs))
        model.compile(loss = 'mse',
                      optimizer = 'adam')

        # fit network
        model.fit(train_x, train_y, epochs = epochs, batch_size = batch_size, verbose = verbose)

        return model

### Forecast Function

As CNN do not have an in-built forecasting function, we have to make it manually.

In [ ]:
# Make a forecast
def forecast(model, history, n_input):
    # flatten data
    data = np.array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))

    # retrieve last observations for input data
    input_x = data[-n_input:, 0]
    
    # reshape into [1, n_input, 1]
    input_x = input_x.reshape((1, len(input_x), 1))
    
    # forecast the next week
    yhat = model.predict(input_x, verbose=0)
  
    # we only want the vector forecast
    yhat = yhat[0]
    
    return yhat

In [ ]:
# Evaluate Forecasting
def evaluate_forecasts(actual, predicted):
    scores = list()
    
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
    
        # calculate mse
        mse = mean_squared_error(actual[:, i], predicted[:, i])

        # calculate rmse
        rmse = np.sqrt(mse)

        # store
        scores.append(rmse)
  
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    
    score = np.sqrt(s / (actual.shape[0] * actual.shape[1]))
            
    return score, scores

In [ ]:
# Evaluate a single model
def evaluate_model(dl_model, train, test, n_input):
    # fit model
    model = build_model(dl_model, train, n_input)
    
    # history is a list of weekly data
    history = [x for x in train]
  
    # walk-forward validation over each week
    predictions = list()
  
    for i in range(len(test)):
        # predict the week
        yhat_sequence = forecast(model, 
                                 history,
                                 n_input)
    
        # store the predictions
        predictions.append(yhat_sequence)
    
        # get real observation and add to history for predicting the next week
        history.append(test[i, :])
  
    # evaluate predictions days for each week
    predictions = np.array(predictions)
    score, scores = evaluate_forecasts(test[:, :, 0], 
                                       predictions)

    return predictions, score, scores

In [ ]:
# Summarize scores
def summarize_scores(name, score, scores):
    s_scores = ', '.join(['%.1f' % s for s in scores])
    
    print('%s: [%.3f] %s' % (name, score, s_scores))

### Result

In [ ]:
# Split into train and test
train, test = split_dataset(bbca_info.values)

# Make into 3D
train, dummy1 = split_sequence(train, 7)
test, dummy2 = split_sequence(test, 7)

In [ ]:
# Evaluate model and get scores
n_input = 7
predictions_cnn, score_cnn, scores_cnn = evaluate_model('CNN', train, test, 7)

# Summarize scores
summarize_scores('CNN', 
                 score_cnn, 
                 scores_cnn)

In [ ]:
# Evaluate model and get scores
n_input = 7
predictions_lstm, score_lstm, scores_lstm = evaluate_model('LSTM', train, test, 7)

# Summarize scores
summarize_scores('LSTM', 
                 score_lstm, 
                 scores_lstm)

In [ ]:
print(f'By using the CNN model, we get that the RMSE is {score_cnn}')
print(f'By using the LSTM model, we get that the RMSE is {score_lstm}')

# Forecasting

**Note**: In this analytics, the prediction and forecasting will only performed on the 'conventional', which is the ARIMA, ARIMAX, and SARIMA model, and will not be performed on the GARCH model.

## Prediction

### ARIMA(4,1,0)

In [ ]:
# Create ARIMA mean forecast
arima_pred = arima_model.get_prediction(start = '2020-04-02')
arima_mean = arima_pred.predicted_mean

# Get confidence intervals of ARIMA mean forecasts
conf_int_arima = arima_pred.conf_int()

# Plot mean ARIMA predictions and observed
plt.figure(figsize = (20,10))

plt.plot(arima_mean.index, 
         arima_mean, 
         label = 'ARIMA(4,1,0)')

plt.fill_between(conf_int_arima.index, 
                 conf_int_arima['lower close'], 
                 conf_int_arima['upper close'], 
                 color = 'pink')

plt.plot(bbca_info['2020-04-02':], 
         label = 'observed')

plt.xlabel('Date')
plt.ylabel('Price (IDR)')
plt.title('ARIMA(4,1,0) on BBCA stock market \'price\' Dataset')
plt.legend()
plt.show()

### ARIMAX(4,1,0,'IDR/USD')

In [ ]:
# ARIMAX

# Create ARIMAX mean forecast
arimax_pred = arimax_model.get_prediction(start = '2020-04-02')
arimax_mean = arimax_pred.predicted_mean

# Get confidence intervals of ARIMAX mean forecasts
conf_int_arimax = arimax_pred.conf_int()

# Plot mean ARIMAX predictions and observed
plt.figure(figsize = (20,10))

plt.plot(arimax_mean.index, 
         arimax_mean, 
         label = 'ARIMAX(4,1,0,\'IDR/USD\')')

plt.fill_between(conf_int_arimax.index, 
                 conf_int_arimax['lower close'], 
                 conf_int_arimax['upper close'],
                 color = 'pink')

plt.plot(bbca_info['2020-04-02':], 
         label = 'observed')

plt.xlabel('Date')
plt.ylabel('Price (IDR)')
plt.title('ARIMAX(4,1,0,\'IDR/USD\') on BBCA stock market \'price\' Dataset')
plt.legend()
plt.show()

### SARIMA(1,1,1)x(0,1,1,12)

In [ ]:
# SARIMA

# Create SARIMA mean forecast
sarima_pred = sarima_model.get_prediction(start = '2020-04-02')
sarima_mean = sarima_pred.predicted_mean

# Get confidence intervals of SARIMA mean forecasts
conf_int_sarima = sarima_pred.conf_int()

# Plot mean ARIMAX predictions and observed
plt.figure(figsize = (20,10))

plt.plot(sarima_mean.index, 
         sarima_mean, 
         label = 'SARIMA(1,1,1)x(0,1,1,12)')

plt.fill_between(conf_int_sarima.index, 
                 conf_int_sarima['lower close'], 
                 conf_int_sarima['upper close'],
                 color = 'pink')

plt.plot(bbca_info['2020-04-02':], 
         label = 'observed')

plt.xlabel('Date')
plt.ylabel('Price (IDR)')
plt.title('SARIMA(1,1,1)x(0,1,1,12) on BBCA stock market \'price\' Dataset')
plt.legend()
plt.show()

### ARIMA(4,1,0)-GARCH(1,1)

In [ ]:
# test_size = len(bbca_info.values)
# arima_garch.forecast(horizon = test_size).variance

### ARIMAX(4,1,0,'IDR/USD')-GARCH(1,1)

## RMSE

In [ ]:
y_bbca = bbca_info['2020-04-02':]

# Prediction for all models
yhat_1_bbca = arima_model.get_prediction(start = '2020-04-02').predicted_mean
yhat_2_bbca = arimax_model.get_prediction(start = '2020-04-02').predicted_mean
yhat_3_bbca = sarima_model.get_prediction(start = '2020-04-02').predicted_mean

# Print MSE
print("MSE for ARIMA(4,1,0) prediction : ", mean_squared_error(y_bbca, yhat_1_bbca))
print("MSE for ARIMAX(4,1,0,'IDR/USD') prediction : ", mean_squared_error(y_bbca, yhat_2_bbca))
print("MSE for SARIMA(1,1,1)x(0,1,1,12) prediction : ", mean_squared_error(y_bbca, yhat_3_bbca))

## Model Summary

In [ ]:
# Cetak Root Mean Square
rmse_1 = np.sqrt(mean_squared_error(y_bbca, yhat_1_bbca))
rmse_2 = np.sqrt(mean_squared_error(y_bbca, yhat_2_bbca))
rmse_3 = np.sqrt(mean_squared_error(y_bbca, yhat_3_bbca))
rmse_4 = score_cnn
rmse_5 = score_lstm

rmse = [rmse_1, rmse_2, rmse_3, rmse_4, rmse_5]

# Cetak Nama Model
model = ['ARIMA(3,1,1)', 'ARIMAX(3,1,1,\'IDR/USD\')', 'SARIMA(1,1,1)x(0,1,1,12)', 
         'Convolutional Neural Network', 'Long Short-Term Memory']

# Buat dictionary
dict_data_bbca = {'Nama Model': model,
                  'RMSE' : rmse}

# Buat DataFramenya
df_score_bbca = pd.DataFrame(dict_data_bbca).sort_values(by = 'RMSE', 
                                                         ascending = True).reset_index(drop = True)

df_score_bbca

### Final Best Model

Based on the AIC-BIC, Diagnostic Test, and also the RMSE results, we can conclude that:

1. LSTM
2. ARIMAX(4,1,0,'IDR/USD')
3. ARIMA(4,1,0)
4. CNN
5. SARIMA(1,1,1)x(0,1,1,12)

respectively, are the **best models** for our BBCA dataset since they are **statistically significance** and have the **lowest AIC-BIC** and **lowest RMSE**.

We will see the forecasted result for each models.

## Forecasting

In [ ]:
def forecast_time_series(data, model_result, n_steps, predicted_date, exog = None, exog_name = None, result = False):
    arimax_pred = model_result.get_forecast(steps = n_steps,
                                            # A week is chosen as forecasting for a long term 
                                            # is not wise and will certainly lead to a high bias/low confidence level
                                            exog = exog_idr_usd[-1:-n_steps-1:-1], 
                                            dynamic = True)

    arimax_mean = pd.DataFrame(arimax_pred.predicted_mean)
    arimax_mean.index = predicted_date
    arimax_mean.columns = ['close']

    arimax_mean = pd.concat([data, arimax_mean], axis = 0)

    # Get confidence intervals of ARIMAX(1,1,0,'IDR/USD') mean forecasts
    conf_int_arimax = arimax_pred.conf_int()
    conf_int_arimax.index = predicted_date


    if result == False:
        # Plot mean ARIMAX predictions and observed
        plt.figure(figsize = (20,10))

        plt.plot(arimax_mean[start_date:], 
                 label = f'Prediction ARIMAX(4,1,0,{exog_name}))', 
                 color = 'red')

        plt.fill_between(conf_int_arimax.index, 
                         conf_int_arimax['lower close'], 
                         conf_int_arimax['upper close'], 
                         label = None,
                         color = 'pink')

        plt.plot(data[start_date:], 
                 label = 'observed', 
                 color = 'blue')

        plt.xlabel('Date')
        plt.ylabel('Price (IDR)')
        plt.title(f'ARIMAX(4,1,0,{exog_name}) on BBCA stock market prediction \'price\' in the next 7 days')
        plt.legend()
        plt.show()
        pass
        
    else:
        conf_int_arimax['close'] = arimax_mean['close'][-n_steps::1]
        conf_int_arimax = conf_int_arimax[['lower close', 'close', 'upper close']]
        return conf_int_arimax

In [ ]:
# Start observing date
start_date = '2021-12-01'

# Start predicted date
x = '2023-03-01'
predicted_dates = pd.date_range(start = dt.datetime.strptime(x, '%Y-%m-%d'), 
                                end = (dt.datetime.strptime(x, '%Y-%m-%d') + dt.timedelta(days = 6)).strftime('%Y-%m-%d'))

In [ ]:
predicted_dates

### ARIMA(4,1,0)

In [ ]:
forecast_time_series(data = bbca_info, 
                     model_result = arima_model, 
                     n_steps = 7, 
                     exog = None, 
                     exog_name = None, 
                     predicted_date = predicted_dates, 
                     result = False)

In [ ]:
# ARIMA(4,1,0) Prediction
result_1 = forecast_time_series(data = bbca_info, 
                                model_result = arima_model, 
                                n_steps = 7, 
                                exog = None, 
                                exog_name = None, 
                                predicted_date = predicted_dates, 
                                result = True)

result_1

### ARIMAX(4,1,0,'IDR/USD')

In [ ]:
forecast_time_series(data = bbca_info, 
                     model_result = arimax_model, 
                     n_steps = 7, 
                     exog = exog_idr_usd, 
                     exog_name = 'IDR/USD', 
                     predicted_date = predicted_dates, 
                     result = False)

In [ ]:
# ARIMAX(4,1,0,'IDR/USD') Prediction
result_2 = forecast_time_series(data = bbca_info, 
                                model_result = arimax_model, 
                                n_steps = 7, 
                                exog = exog_idr_usd, 
                                exog_name = 'IDR/USD', 
                                predicted_date = predicted_dates, 
                                result = True)

result_2

### SARIMA(1,1,1)x(0,1,1,12)

In [ ]:
forecast_time_series(data = bbca_info, 
                     model_result = sarima_model, 
                     n_steps = 7, 
                     exog = None, 
                     exog_name = None, 
                     predicted_date = predicted_dates, 
                     result = False)

In [ ]:
# SARIMA(1,1,1)x(0,1,1,12) Prediction
result_3 = forecast_time_series(data = bbca_info, 
                                model_result = sarima_model, 
                                n_steps = 7, 
                                exog = None, 
                                exog_name = None, 
                                predicted_date = predicted_dates, 
                                result = True)

result_3

**Convolutional Neural Network**

In [ ]:
predictions_cnn[-1]

**Long Short-Term Memory**

In [ ]:
predictions_lstm[-1]

## Conclusion

Based on our model, we can see that for the **next 7 days**, we can expect that the **BBCA stock market price** will be relatively stagnant at 8790 IDR with a margin error of ~4.47%.

## Recommendation

**Technical**

---

Consider the **ARIMA-GARCH** and **ARIMAX-GARCH** model that has been previously calculated. The model is **not used** here since the author still does not find a **valid resource** as to **combine** the result from ARIMA and GARCH model separately in Python, unlike **rugarch** in **R**.

**Non-technical**

---

The BBCA stock market price do not show any sign of dropping anytime soon. So, if you are a **shareholder**, try **not to sell your stock**, except in case of emergencies. And for a **non-shareholder**, you probably want to **wait** when the price is showing a **decrement trend**, which is **probably** not in the near future (1-week time).

# Portfolio Optimization

Since the author is still an Undergraduate student who has only learn basic optimization method on portfolio, thus we will only conduct simple **Markowitz Frontier Model**.

## Data Preparation

In [ ]:
portfolio = pd.DataFrame()

portfolio.index = bbca_info.index

# Get BBCA and ^JKSE close value
portfolio['BBCA'] = bbca_info.values
jkse_info = jkse_info['2020-04-01':]
portfolio['^JKSE'] = jkse_info.values

portfolio

In [ ]:
returns = portfolio / portfolio.shift(1)
returns

## Sharpe Ratio

In [ ]:
n_portfolios = 10000
weight = np.zeros((n_portfolios, 2))

expectedReturn = np.zeros(n_portfolios)
expectedVolatility = np.zeros(n_portfolios)
sharpeRatio = np.zeros(n_portfolios)

meanReturn = returns.mean()
S = returns.cov()
for k in range(n_portfolios):
    # Generate random weights for each portfolio
    w = np.array(np.random.random(2))
    w = w / np.sum(w)
    weight[k, :] = w
    
    # Expected return & volatility
    expectedReturn[k] = np.sum(meanReturn * w)
    expectedVolatility[k] = np.sqrt(np.matmul(w.T, np.matmul(S, w)))
    
    # Sharpe Ratio
    sharpeRatio[k] = expectedReturn[k] / expectedVolatility[k]

In [ ]:
maxIdx = sharpeRatio.argmax()
optWeight = weight[maxIdx, :]

maxIdx, optWeight

## Return vs Volatility

In [ ]:
plt.figure(figsize = (12, 8))

plt.scatter(expectedVolatility, expectedReturn, c = sharpeRatio)
plt.scatter(expectedVolatility[maxIdx], expectedReturn[maxIdx])
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.title('Return vs Volatility')
plt.colorbar(label = 'SR')

plt.show()

## Markowitz Frontier

In [ ]:
def negativeSR(w):
    w = np.array(w)
    R = np.sum(meanReturn * w)
    V = np.sqrt(np.matmul(w.T, np.matmul(S, w)))
    
    return -R / V

def checkSumToOne(w):
    return np.sum(w) - 1

w0 = [0.5, 0.5]
bounds = ((0, 1), (0, 1))
constraints = ({'type': 'eq',
                'fun': lambda x: np.sum(x) - 1})
w_opt = minimize(negativeSR, 
                 w0, 
                 method = 'SLSQP', 
                 bounds = bounds,
                 constraints = constraints)

w_opt

In [ ]:
def minimizeVolatility(w):
    w = np.array(w)
    
    return np.sqrt(np.matmul(w.T, np.matmul(S, w)))

def getReturn(w):
    w = np.array(w)
    
    return np.sum(meanReturn * w)

w0 = [0.5, 0.5]
bounds = ((0, 1), (0, 1))
return_lst = np.linspace(min(expectedReturn), max(expectedReturn), 100)
volatility_opt = []

for R in return_lst:
    constraints = ({'type': 'eq',
                    'fun': lambda x: np.sum(x) - 1},
                   {'type': 'eq',
                    'fun': lambda w: getReturn(w) - R})
    w_opt = minimize(minimizeVolatility, 
                     w0, 
                     method = 'SLSQP', 
                     bounds = bounds,
                     constraints = constraints)
    
    volatility_opt.append(w_opt['fun'])

In [ ]:
def efficientFrontierPlot():
    plt.figure(figsize = (12, 8))

    plt.scatter(expectedVolatility, expectedReturn, c = sharpeRatio)
    plt.scatter(expectedVolatility[maxIdx], expectedReturn[maxIdx])
    plt.plot(volatility_opt, return_lst)
    plt.xlabel('Expected Volatility')
    plt.ylabel('Expected Return')
    plt.title('Return vs Volatility')
    plt.colorbar(label = 'SR')

    plt.show()

    optReturn = expectedReturn[maxIdx]
    optVolatility = expectedVolatility[maxIdx]
    optRatio = sharpeRatio[maxIdx]

    print('Expected Annual Return', optReturn)
    print('Annual Volatility', optVolatility)
    print('Sharpe Ratio', optRatio)

efficientFrontierPlot()

Notice that the Efficient Frontier is at the **left edge** of the curve, since it shows the **maximum return** we can get, with the **least possible risk**. The curve is also relatively **dense** because if we notice that BBCA.JK and ^JKSE almost have the **same trend**, which means that both have relatively same daily Expected Return. Thus, the author argue that there does not much difference between investing in BBCA.JK or ^JKSe stock market. But, this method suggests us to **invest heavily** in **^JKSE**, because as we can see that ^JKSE has **lower daily jump** between the prices, which means that its **less risky** for us to invest there, with relatively the **same return**.